In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('../../smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V


/tmp/ipykernel_31520/786496249.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp


3.0.0


In [2]:
BSMlist = getParticlesFromSLHA('TRPS1/TRPS1_100_1_1.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)


[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, xr, xc, xc~, xd, xd~, y0]


In [3]:
slhaFolder = 'TRPS1'
resultsFolder = 'TRPS1_results'
File = 'parameters.ini'
!chmod -R +rwx {slhaFolder}


In [4]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}


INFO in databaseObj.loadBinaryFile() in 551: loading binary db file /home/yoxara/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 558: Loaded database from /home/yoxara/smodels-database/db3.pcl in 8.3 secs.
INFO in modelTester.loadDatabaseResults() in 508: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 107 files with a single process. Messages will be redirected to smodels.log


In [5]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)


In [6]:
print(len(data))


107


In [7]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)
smodelsDF = json_normalize(data)
smodelsDF


,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,TRPS1_800_200_200.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",1.315767e+02,0.000000e+00,1.315767e+02,8.621335e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_800_200_200.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRPS1_900_350_350.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",7.291310e+01,0.000000e+00,7.291310e+01,3.387690e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_900_350_350.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,TRPS1_250_1_1.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRPS1/TRPS1_250_1_1.slha,3.0.0,3.0.0,0.000000e+00,1.115700e+04,1.163030e+04,NaN,[TRPS1],"[(y0, 250.0), (xd~, 1.0), (xd, 1.0)]",ATLAS-EXOT-2018-06,NaN,1.300000e+01,1.390000e+02,upperLimit,9.593046e-01,NaN,"[(y0, 9.946866), (xd~, stable), (xd, stable)]"
106,TRPS1_950_300_300.slha,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",4.899579e+01,0.000000e+00,4.899579e+01,2.783721e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_950_300_300.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >=52])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 54])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)


In [9]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)


In [10]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')


In [11]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names


Final number of data points: 107


In [12]:
#Save DataFrame to pickle file:
dataDF.to_csv('TRPS1.csv')
print(len(dataDF))
dataDF


107


,filename,mass.52,mass.54,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.54,width.5000001,width.9000002,width.9000003,width.9000004,extpar.1,extpar.2,extpar.3,extpar.4,extpar.5,extpar.6,extpar.7,extpar.8,extpar.9,extpar.10,extpar.11,extpar.12,extpar.13,extpar.14,extpar.15,extpar.16,extpar.17,extpar.18,extpar.19,extpar.20,extpar.21,extpar.22,extpar.23,extpar.24,extpar.25,xsec13TeV(fb).54,"BRs.y0.t-,t+","BRs.y0.xd~,xd",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,TRPS1_800_200_200.slha,2.000000e+02,8.000000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,6.963766e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.177900e+02,6.041446e-01,3.958554e-01,2.177900e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",1.315767e+02,0.000000e+00,1.315767e+02,8.621335e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_800_200_200.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRPS1_900_350_350.slha,3.500000e+02,9.000000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,7.095160e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.067900e+02,6.827709e-01,3.172291e-01,1.067900e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",7.291310e+01,0.000000e+00,7.291310e+01,3.387690e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_900_350_350.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,TRPS1_250_1_1.slha,1.000000e+00,2.500000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,9.946866e+00,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.0

In [13]:
dataDF['totalxsec13TeV(fb)']*(dataDF['BRs.y0.xd~,xd'])



0     8.621335e+01
1     3.387690e+01
          ...     
105   1.115700e+04
106   2.783721e+01
Length: 107, dtype: float64

In [14]:
dataDF['ExptRes.result0.theory prediction (fb)']


0              NaN
1              NaN
          ...     
105   1.115700e+04
106            NaN
Name: ExptRes.result0.theory prediction (fb), Length: 107, dtype: float64

In [15]:
#(dataDF['xsec13TeV(fb).55'] *(3 *dataDF['BRs.y1.q,q'] + dataDF['BRs.y1.c,c'] + dataDF['BRs.y1.b,b'] ))/dataDF['ExptRes.result0.theory prediction (fb)']


In [16]:
mask = dataDF['ExptRes.result0.r'].isna()
rows_with_nan = dataDF[mask]
rows_with_nan
#print(rows_with_nan)


,filename,mass.52,mass.54,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.54,width.5000001,width.9000002,width.9000003,width.9000004,extpar.1,extpar.2,extpar.3,extpar.4,extpar.5,extpar.6,extpar.7,extpar.8,extpar.9,extpar.10,extpar.11,extpar.12,extpar.13,extpar.14,extpar.15,extpar.16,extpar.17,extpar.18,extpar.19,extpar.20,extpar.21,extpar.22,extpar.23,extpar.24,extpar.25,xsec13TeV(fb).54,"BRs.y0.t-,t+","BRs.y0.xd~,xd",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,TRPS1_800_200_200.slha,2.000000e+02,8.000000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,6.963766e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.177900e+02,6.041446e-01,3.958554e-01,2.177900e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",1.315767e+02,0.000000e+00,1.315767e+02,8.621335e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_800_200_200.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRPS1_900_350_350.slha,3.500000e+02,9.000000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,7.095160e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.067900e+02,6.827709e-01,3.172291e-01,1.067900e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",7.291310e+01,0.000000e+00,7.291310e+01,3.387690e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_900_350_350.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,TRPS1_450_200_200.slha,2.000000e+02,4.500000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,2.510134e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00

In [17]:
mask = dataDF['ExptRes.result0.r'].isna()
rows_with_nan = dataDF[mask]
#rows_with_nan.to_csv('rows_with_nan_.csv', index=False)
rows_with_nan


,filename,mass.52,mass.54,mass.5000001,mass.9000002,mass.9000003,mass.9000004,width.54,width.5000001,width.9000002,width.9000003,width.9000004,extpar.1,extpar.2,extpar.3,extpar.4,extpar.5,extpar.6,extpar.7,extpar.8,extpar.9,extpar.10,extpar.11,extpar.12,extpar.13,extpar.14,extpar.15,extpar.16,extpar.17,extpar.18,extpar.19,extpar.20,extpar.21,extpar.22,extpar.23,extpar.24,extpar.25,xsec13TeV(fb).54,"BRs.y0.t-,t+","BRs.y0.xd~,xd",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,TRPS1_800_200_200.slha,2.000000e+02,8.000000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,6.963766e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.177900e+02,6.041446e-01,3.958554e-01,2.177900e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",1.315767e+02,0.000000e+00,1.315767e+02,8.621335e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_800_200_200.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRPS1_900_350_350.slha,3.500000e+02,9.000000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,7.095160e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.067900e+02,6.827709e-01,3.172291e-01,1.067900e+02,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > inv)', 'Masses (GeV)'...",7.291310e+01,0.000000e+00,7.291310e+01,3.387690e+01,0.000000e+00,5,2.000000e-01,1,TRPS1/TRPS1_100_1_1.slha,5.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,0,Input file ok,TRPS1/TRPS1_900_350_350.slha,3.0.0,3.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,TRPS1_450_200_200.slha,2.000000e+02,4.500000e+02,1.000000e+01,9.118760e+01,7.982436e+01,7.982436e+01,2.510134e+01,0.000000e+00,2.495200e+00,2.085000e+00,2.085000e+00